In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import zipfile

In [2]:
def load_data_year(year, cols):
    file = f'../../../microdados_anos/MICRODADOS_ENEM_{year}.csv'
    df = pd.read_csv(file, encoding='cp1252', sep=';', usecols=cols, nrows=2000)
    
    return df

In [3]:
cols_used = ['NU_INSCRICAO', 'NU_ANO', 'NO_MUNICIPIO_RESIDENCIA', 'SG_UF_RESIDENCIA',
             'NU_IDADE', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA',
             'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA',
             'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
             'TP_PRESENCA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC',
             'NU_NOTA_MT', 'NU_NOTA_COMP1',
             'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5',
             'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006',
             'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014',
             'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022',
             'Q023', 'Q024', 'Q025']

In [4]:
def union_datas(years, cols):
    df_final = pd.DataFrame([], columns=cols)
    
    for y in years:
        df_final = pd.concat([df_final, load_data_year(y, cols)])
    
    return df_final

In [5]:
years = [2016, 2017, 2018, 2019]

In [6]:
%%time

df_union = union_datas(years, cols_used)

Wall time: 160 ms


In [7]:
df_union.head()

,NU_INSCRICAO,NU_ANO,NO_MUNICIPIO_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,160000301384,2016,João Pessoa,PB,23,M,0.0,3,1,7,...,A,A,A,C,B,B,C,B,B,B
1,160000000001,2016,Vitorino,PR,20,M,0.0,1,1,3,...,B,A,A,B,B,A,C,B,B,B
2,160000000002,2016,Salvador,BA,21,M,0.0,3,1,1,...,A,A,A,B,A,A,C,B,B,B
3,160000000003,2016,Belém,PA,17,M,0.0,1,1,1,...,A,A,A,B,B,A,B,A,A,B
4,160000000004,2016,Brasília,DF,36,F,2.0,1,1,10,...,A,A,A,A,A,A,A,A,A,A


# Pre-processing

In [8]:
pd.DataFrame({'types': df_union.dtypes, 'missing': df_union.isna().sum()})

,types,missing
NU_INSCRICAO,object,0
NU_ANO,object,0
NO_MUNICIPIO_RESIDENCIA,object,0
SG_UF_RESIDENCIA,object,0
NU_IDADE,object,0
TP_SEXO,object,0
TP_ESTADO_CIVIL,float64,267
TP_COR_RACA,object,0
TP_ST_CONCLUSAO,object,0
TP_ANO_CONCLUIU,object,0


In [9]:
df_final = df_union

In [10]:
imp_ec = SimpleImputer(strategy="most_frequent")
imp_ec.fit(df_final[['TP_ESTADO_CIVIL']])

SimpleImputer(strategy='most_frequent')

In [11]:
df_final['TP_ESTADO_CIVIL'] = imp_ec.transform(df_final[['TP_ESTADO_CIVIL']])

In [12]:
df_groups = df_final.groupby(['TP_ESCOLA', 'TP_ST_CONCLUSAO', 'SG_UF_RESIDENCIA']).mean().iloc[:,1:]
df_groups = df_groups.rename_axis(['TP_ESCOLA', 'TP_ST_CONCLUSAO', 'SG_UF_RESIDENCIA']).reset_index()

In [13]:
df_groups.head()

,TP_ESCOLA,TP_ST_CONCLUSAO,SG_UF_RESIDENCIA,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO
0,1,1,AC,501.509091,559.618182,529.920000,481.410000,124.000000,128.000000,120.000000,120.000000,96.000000,588.000000
1,1,1,AL,489.774074,540.514286,513.432143,513.105556,124.285714,127.500000,120.000000,127.857143,92.500000,592.142857
2,1,1,AM,479.650000,538.665385,516.798077,498.227083,123.846154,111.538462,110.769231,115.769231,70.769231,532.692308
3,1,1,AP,481.591304,510.780000,505.212000,485.478261,109.600000,107.200000,100.000000,108.800000,76.000000,501.600000
4,1,1,BA,497.883571,541.560046,525.986073,509.273810,123.561644,117.716895,111.278539,119.908676,84.383562,556.849315


In [14]:
def add_notas(x):
    if(sum(pd.isna(x)) > 0):
        df_x = df_groups.query('TP_ESCOLA==@x.TP_ESCOLA & TP_ST_CONCLUSAO==@x.TP_ST_CONCLUSAO & SG_UF_RESIDENCIA==@x.SG_UF_RESIDENCIA')
        
        for c in df_x.columns:
            if(pd.isna(x[c]) > 0):
                x[c] = df_x[c]
    
    return x

In [18]:
df_final = df_final.apply(add_notas, axis=1)

In [19]:
pd.DataFrame({'types': df_final.dtypes, 'missing': df_final.isna().sum()})

,types,missing
NU_INSCRICAO,int64,0
NU_ANO,int64,0
NO_MUNICIPIO_RESIDENCIA,object,0
SG_UF_RESIDENCIA,object,0
NU_IDADE,int64,0
TP_SEXO,object,0
TP_ESTADO_CIVIL,float64,0
TP_COR_RACA,int64,0
TP_ST_CONCLUSAO,int64,0
TP_ANO_CONCLUIU,int64,0


In [17]:
%%time

# Save
#file = '../datasets/integrated_data.csv'
#df_final.to_csv(file, index=False)

#z = zipfile.ZipFile('../datasets/integrated_data.zip', 'w', zipfile.ZIP_DEFLATED)
#z.write(file)    
#z.close()

Wall time: 0 ns
